# Healthcare Hack Nights: Part I

In [ ]:
## RNA-Seq
Questions that can be answered by RNA-seq:
    - What genes are differentially expressed between group samples?
    - How does gene expression change across time or conditions? (eg, in benign vs malignant tumors)
    - What pathways or processes are enriched under a condition?

In [151]:
%matplotlib inline

In [15]:
import pandas as pd

In [30]:
lihc = pd.read_csv('../lihc_rnaseq.csv')
lihc.shape

(423, 20532)

In [18]:
lihc.head()

,bcr_patient_barcode,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
0,TCGA-2V-A95S-01A-11R-A37K-07,0.0,1.5051,3.7074,90.1124,1017.1038,0.0,141.3911,0.6516,0.0000,...,24.7597,273.6602,794.2662,18.2440,499.1041,3172.5037,890.0472,510.1808,3.9094,6.5157
1,TCGA-2Y-A9GS-01A-12R-A38B-07,0.0,26.4120,2.6663,71.0054,639.2311,0.0,122.7206,1.4786,0.0000,...,68.5067,632.8241,1153.7703,71.4638,1000.4929,5301.1336,755.5446,860.5224,6.4071,482.9966
2,TCGA-2Y-A9GT-01A-11R-A38B-07,0.0,0.0000,4.4833,95.5122,742.4344,0.0,95.0460,1.7933,0.8967,...,46.6263,1219.4575,1133.3782,12.5532,1289.3970,3219.0092,860.7935,523.6494,14.3466,83.3894
3,TCGA-2Y-A9GU-01A-11R-A38B-07,0.0,5.7222,5.1216,61.6679,1186.9807,0.0,280.2709,0.8341,0.0000,...,18.3511,285.2758,1150.2786,9.1755,941.7437,3092.9899,1339.6283,343.6655,2.5024,2.5024
4,TCGA-2Y-A9GV-01A-11R-A38B-07,0.0,11.4975,5.4230,104.4670,878.1726,0.0,282.5719,0.0000,0.0000,...,41.4552,999.1540,1631.9797,4.2301,1380.7107,2902.7073,575.2961,665.8206,2.5381,119.2893


The dataset contains the number of counts for ~20k genes defined by their Entrez transcript ID x 423 deidentified patients. 

Entrez (https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html) is a data retrieval system that provides users access to NCBI’s databases such as PubMed, GenBank, GEO, and many others. You can access Entrez from a web browser to manually enter queries, or you can use Biopython’s Bio.Entrez module for programmatic access to Entrez. 

Entrez gene IDs are unique gene identifiers that can be used to trace a particular gene or transcript to the genome.

In [19]:
# Get Entrez transcript IDs

ids = pd.Series(lihc.columns.values[1:]).apply(lambda x: x.split('|')[1]).values
ids[:5]

array(['100130426', '100133144', '100134869', '10357', '10431'],
      dtype=object)

# BioMart


In [2]:
from pybiomart import Server, Dataset

In [9]:
# Retrieving a dataset directly with known dataset name

dataset = Dataset(name='hsapiens_gene_ensembl',
                  host='http://www.ensembl.org')

dataset.query(
              filters={'chromosome_name': ['1','2']})

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version
0,ENSG00000200036,ENSG00000200036.1,ENST00000363166,ENST00000363166.1
1,ENSG00000252396,ENSG00000252396.1,ENST00000516587,ENST00000516587.1
2,ENSG00000252429,ENSG00000252429.2,ENST00000516620,ENST00000516620.2
3,ENSG00000221643,ENSG00000221643.1,ENST00000408716,ENST00000408716.1
4,ENSG00000264371,ENSG00000264371.1,ENST00000580572,ENST00000580572.1
...,...,...,...,...
36496,ENSG00000196290,ENSG00000196290.15,ENST00000409357,ENST00000409357.5
36497,ENSG00000196290,ENSG00000196290.15,ENST00000409129,ENST00000409129.2
36498,ENSG00000196290,ENSG00000196290.15,ENST00000409588,ENST00000409588.1
36499,ENSG00000196290,ENSG00000196290.15,ENST00000436412,ENST00000436412.1


The `attributes` attribute can be used to pull up a list of additional fields available from the dataset

In [7]:
list(dataset.attributes)

['ensembl_gene_id',
 'ensembl_gene_id_version',
 'ensembl_transcript_id',
 'ensembl_transcript_id_version',
 'ensembl_peptide_id',
 'ensembl_peptide_id_version',
 'ensembl_exon_id',
 'description',
 'chromosome_name',
 'start_position',
 'end_position',
 'strand',
 'band',
 'transcript_start',
 'transcript_end',
 'transcription_start_site',
 'transcript_length',
 'transcript_tsl',
 'transcript_gencode_basic',
 'transcript_appris',
 'transcript_mane_select',
 'external_gene_name',
 'external_gene_source',
 'external_transcript_name',
 'external_transcript_source_name',
 'transcript_count',
 'percentage_gene_gc_content',
 'gene_biotype',
 'transcript_biotype',
 'source',
 'transcript_source',
 'version',
 'transcript_version',
 'peptide_version',
 'phenotype_description',
 'Source_name',
 'study_external_id',
 'strain_name',
 'strain_gender',
 'p_value',
 'go_id',
 'name_1006',
 'definition_1006',
 'go_linkage_type',
 'namespace_1003',
 'goslim_goa_accession',
 'goslim_goa_description',


We can map the gene stable ID with the mappings from the to get a sense of which pathways are linked to a particular gene:

In [21]:
dataset.filters

{'link_so_mini_closure': <biomart.Filter name='link_so_mini_closure', type='list'>,
 'link_go_closure': <biomart.Filter name='link_go_closure', type='text'>,
 'link_ensembl_transcript_stable_id': <biomart.Filter name='link_ensembl_transcript_stable_id', type='text'>,
 'gene_id': <biomart.Filter name='gene_id', type='text'>,
 'transcript_id': <biomart.Filter name='transcript_id', type='text'>,
 'link_ensembl_gene_id': <biomart.Filter name='link_ensembl_gene_id', type='text'>,
 'chromosome_name': <biomart.Filter name='chromosome_name', type='text'>,
 'start': <biomart.Filter name='start', type='text'>,
 'end': <biomart.Filter name='end', type='text'>,
 'band_start': <biomart.Filter name='band_start', type='drop_down_basic_filter'>,
 'band_end': <biomart.Filter name='band_end', type='drop_down_basic_filter'>,
 'marker_start': <biomart.Filter name='marker_start', type='drop_down_basic_filter'>,
 'marker_end': <biomart.Filter name='marker_end', type=''>,
 'hsapiens_encode.type': <biomart.Fi

In [86]:
ensmbl_entrez_gene_ids = dataset.query(attributes=['ensembl_transcript_id', 'entrezgene_id'])
ensmbl_entrez_gene_ids.tail(10)

,Transcript stable ID,NCBI gene ID
251176,ENST00000644207,NaN
251177,ENST00000647544,NaN
251178,ENST00000642596,NaN
251179,ENST00000643537,NaN
251180,ENST00000644633,NaN
251181,ENST00000642800,NaN
251182,ENST00000645112,56169.0
251183,ENST00000642712,56169.0
251184,ENST00000646090,56169.0
251185,ENST00000643960,56169.0


In [88]:
ensmbl_entrez_gene_ids.dropna(inplace=True)
ensmbl_entrez_gene_ids['NCBI gene ID'] = ensmbl_entrez_gene_ids['NCBI gene ID'].astype(int)
ensmbl_entrez_gene_ids = ensmbl_entrez_gene_ids.set_index('NCBI gene ID').to_dict()['Transcript stable ID']
ensmbl_entrez_gene_ids

{4535: 'ENST00000361390',
 4536: 'ENST00000361453',
 4512: 'ENST00000361624',
 113219467: 'ENST00000387416',
 4513: 'ENST00000361739',
 4509: 'ENST00000361851',
 4508: 'ENST00000361899',
 4514: 'ENST00000362079',
 4537: 'ENST00000361227',
 4539: 'ENST00000361335',
 4538: 'ENST00000361381',
 4540: 'ENST00000361567',
 4541: 'ENST00000361681',
 4519: 'ENST00000361789',
 1028: 'ENST00000471157',
 51621: 'ENST00000616962',
 255027: 'ENST00000567442',
 9665: 'ENST00000549219',
 4849: 'ENST00000611667',
 102723475: 'ENST00000622690',
 9093: 'ENST00000431375',
 79165: 'ENST00000619669',
 57348: 'ENST00000423529',
 1113: 'ENST00000556876',
 5165: 'ENST00000493226',
 89927: 'ENST00000565913',
 94104: 'ENST00000464256',
 1007: 'ENST00000511822',
 147798: 'ENST00000620520',
 23532: 'ENST00000398741',
 338755: 'ENST00000338569',
 144125: 'ENST00000307401',
 54664: 'ENST00000462754',
 51277: 'ENST00000534855',
 440243: 'ENST00000612056',
 161725: 'ENST00000560598',
 2558: 'ENST00000400081',
 285148:

In [91]:
ensmbl_ids = pd.Series(lihc.columns.values[1:]).apply(lambda x: x.split('|')[1]).astype(int).map(ensmbl_entrez_gene_ids).dropna()

# Had to drop ~2k that didn't align, is there a better way?

In [80]:
dataset.filters

{'link_so_mini_closure': <biomart.Filter name='link_so_mini_closure', type='list'>,
 'link_go_closure': <biomart.Filter name='link_go_closure', type='text'>,
 'link_ensembl_transcript_stable_id': <biomart.Filter name='link_ensembl_transcript_stable_id', type='text'>,
 'gene_id': <biomart.Filter name='gene_id', type='text'>,
 'transcript_id': <biomart.Filter name='transcript_id', type='text'>,
 'link_ensembl_gene_id': <biomart.Filter name='link_ensembl_gene_id', type='text'>,
 'chromosome_name': <biomart.Filter name='chromosome_name', type='text'>,
 'start': <biomart.Filter name='start', type='text'>,
 'end': <biomart.Filter name='end', type='text'>,
 'band_start': <biomart.Filter name='band_start', type='drop_down_basic_filter'>,
 'band_end': <biomart.Filter name='band_end', type='drop_down_basic_filter'>,
 'marker_start': <biomart.Filter name='marker_start', type='drop_down_basic_filter'>,
 'marker_end': <biomart.Filter name='marker_end', type=''>,
 'hsapiens_encode.type': <biomart.Fi

In [96]:
# Find a faster way to do this

attributes = [
#     'gene_id',
    'entrezgene_id',
    'ensembl_gene_id',
    'ensembl_transcript_id',
    'go_id',
    'name_1006',
    'definition_1006',
    'go_linkage_type',
    'hgnc_id',
    'hgnc_symbol',
#     'hgnc_trans_name',
]
go_mappings = dataset.query(attributes=attributes)

In [97]:
go_mappings.to_csv('data/go_mappings.csv', index=False)

In [92]:
# Figure out better way to do this
attributes = [
#     'gene_id',
    'entrezgene_id',
    'ensembl_gene_id',
    'ensembl_transcript_id',
    'go_id',
    'name_1006',
    'definition_1006',
    'go_linkage_type',
    'hgnc_id',
    'hgnc_symbol',
#     'hgnc_trans_name',
]
dataset.query(attributes=attributes,
              filters={'transcript_id': ensmbl_ids.values}
             )

,NCBI gene ID,Gene stable ID,Transcript stable ID,GO term accession,GO term name,GO term definition,GO term evidence code,HGNC ID,HGNC symbol


## Examine Gene Counts



In [98]:
go_mappings = pd.read_csv('data/go_mappings.csv')

go_mappings.shape

/Users/mattheweng/Desktop/Galvanize/Projects/hacknights/env-tcga/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1392800, 9)

In [99]:
go_mappings.head()

,NCBI gene ID,Gene stable ID,Transcript stable ID,GO term accession,GO term name,GO term definition,GO term evidence code,HGNC ID,HGNC symbol
0,NaN,ENSG00000210049,ENST00000387314,NaN,NaN,NaN,NaN,HGNC:7481,MT-TF
1,NaN,ENSG00000211459,ENST00000389680,NaN,NaN,NaN,NaN,HGNC:7470,MT-RNR1
2,NaN,ENSG00000210077,ENST00000387342,NaN,NaN,NaN,NaN,HGNC:7500,MT-TV
3,NaN,ENSG00000210082,ENST00000387347,NaN,NaN,NaN,NaN,HGNC:7471,MT-RNR2
4,NaN,ENSG00000209082,ENST00000386347,NaN,NaN,NaN,NaN,HGNC:7490,MT-TL1


In [104]:
go_mappings.loc[~go_mappings['NCBI gene ID'].isnull()]

,NCBI gene ID,Gene stable ID,Transcript stable ID,GO term accession,GO term name,GO term definition,GO term evidence code,HGNC ID,HGNC symbol
5,4535,ENSG00000198888,ENST00000361390,GO:0016020,membrane,A lipid bilayer along with all the proteins an...,IEA,HGNC:7455,MT-ND1
6,4535,ENSG00000198888,ENST00000361390,GO:0016021,integral component of membrane,The component of a membrane consisting of the ...,IEA,HGNC:7455,MT-ND1
7,4535,ENSG00000198888,ENST00000361390,GO:0055114,oxidation-reduction process,A metabolic process that results in the remova...,IEA,HGNC:7455,MT-ND1
8,4535,ENSG00000198888,ENST00000361390,GO:0005743,mitochondrial inner membrane,"The inner, i.e. lumen-facing, lipid bilayer of...",IEA,HGNC:7455,MT-ND1
9,4535,ENSG00000198888,ENST00000361390,GO:0005739,mitochondrion,"A semiautonomous, self replicating organelle t...",IEA,HGNC:7455,MT-ND1
...,...,...,...,...,...,...,...,...,...
1392787,60678,ENSG00000284869,ENST00000646013,NaN,NaN,NaN,NaN,HGNC:24614,EEFSEC
1392796,56169,ENSG00000285114,ENST00000645112,NaN,NaN,NaN,NaN,HGNC:7151,GSDMC
1392797,56169,ENSG00000285114,ENST00000642712,NaN,NaN,NaN,NaN,HGNC:7151,GSDMC
1392798,56169,ENSG00000285114,ENST00000646090,NaN,NaN,NaN,NaN,HGNC:7151,GSDMC


In [142]:
entrez_ids = list(pd.Series(lihc.columns.values[1:]).apply(lambda x: x.split('|')[1]).values)
entrez_ids = list(set(pd.Series(entrez_ids).astype(int).values))
entrez_ids

[1,
 2,
 131076,
 9,
 10,
 12,
 13,
 14,
 15,
 16,
 26054,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 163882,
 43,
 319139,
 131118,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 60,
 6525,
 83935,
 70,
 71,
 72,
 131149,
 81,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 163933,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 131177,
 107,
 108,
 109,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 6536,
 123,
 124,
 125,
 126,
 127,
 128,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 196740,
 196743,
 140,
 141,
 142,
 143,
 146,
 147,
 148,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 84671,
 149830,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 6548,
 181,
 182,
 183,
 196792,
 185,
 186,
 187,
 6549,
 189,
 190,
 191,
 6550,
 89882,
 196,
 197,
 199,
 202,
 203,
 204,
 205,
 164045,
 207,
 208,
 6553,
 210,
 211,
 2

In [136]:
# Get list of transcript ids from GO mapping
transcript_ids_mapping = go_mappings['NCBI gene ID'].dropna().unique()
def check_type(x):
    try:
        
        return int(x)
    except:
        return -1

transcript_ids_mapping = list(map(lambda x: check_type(x), transcript_ids_mapping))

In [148]:
len(set(transcript_ids_mapping).intersection(set(entrez_ids)))

18742

In [150]:
lihc[1:].sum()

bcr_patient_barcode    TCGA-2Y-A9GS-01A-12R-A38B-07TCGA-2Y-A9GT-01A-1...
?|100130426                                                      21.0025
?|100133144                                                      1870.33
?|100134869                                                      1948.12
?|10357                                                          40000.7
                                             ...                        
ZYX|7791                                                     1.59719e+06
ZZEF1|23140                                                       294653
ZZZ3|26009                                                        242955
psiTPTE22|387590                                                 3631.78
tAKR|389932                                                      39635.7
Length: 20532, dtype: object

In [ ]:
df_genes <- df[,df_colnames_genes %in% go_map$hgnc_symbol]
colnames(df_genes) <- df_colnames_genes[which(df_colnames_genes %in% go_map$hgnc_symbol)]
df_genes %>% dim()

## TCGA
For the RNA-Seq analysis workflow, we obtain the raw FASTQ sequencing files from the sequencing facility. We assess the quality of our sequence reads for each sample, then determine from where on the genome the reads originated by performing alignment. We will use the information about where the reads align to generate the count matrix, which we will be using to start the differential expression analysis.



In [ ]:
Differential gene expression analysis
- https://genomebiology.biomedcentral.com/articles/10.1186/s13059-016-0881-8
- http://bioconductor.org/packages/release/bioc/html/DESeq.htmlb
- https://nbviewer.jupyter.org/github/maayanlab/Zika-RNAseq-Pipeline/blob/master/Zika.ipynbb

In [ ]:
## Load expression matrix
expr_df = pd.read_csv(os.path.join(os.environ['WORKDIR'], 'repCpmMatrix_featureCounts.csv'))
expr_df = expr_df.set_index(expr_df.columns[0])
expr_df.head()

In [ ]:
## Filter out non-expressed genes
expr_df = expr_df.loc[expr_df.sum(axis=1) > 0, :]
print expr_df.shape

## Filter out lowly expressed genes
mask_low_vals = (expr_df > 0.3).sum(axis=1) > 2
expr_df = expr_df.loc[mask_low_vals, :]

print expr_df.shape

In [ ]:
meta_df = pd.read_csv(os.path.join(os.environ['WORKDIR'], 'SraRunTable.txt'), sep='\t').set_index('Run_s')
print meta_df.shape
# re-order the index to make it the same with expr_df
meta_df = meta_df.ix[expr_df.columns]
meta_df

## PCA Plots
- Generate PCA plots to observe whether samples cluster as expected (controls with controls and treatments with treatments)

In [ ]:
import RNAseq

In [ ]:
# plot PCA
%matplotlib inline
RNAseq.PCA_plot(expr_df.values, meta_df['infection_status_s'], 
         standardize=2, log=True, 
         show_text=False, sep=' ', legend_loc='upper right')

## Differential Gene Expression
Now we are ready to identify the differentially expressed genes between the two sets of samples: control vs. treatment. We will achieve this using the Characteristic Direction method[6](#ref6) that we developed and published in BMC Bioinformatics in 2014.

An implementation in Python of the Characteristic Direction method can be downloaded and installed from here: https://github.com/wangz10/geode.
    


In [ ]:
import geode
d_platform_cd = {} # to top up/down genes
cd_results = pd.DataFrame(index=expr_df.index)

sample_classes = {}
for layout in meta_df['LibraryLayout_s'].unique():
    ## make sample_class 
    sample_class = np.zeros(expr_df.shape[1], dtype=np.int32)
    sample_class[meta_df['LibraryLayout_s'].values == layout] = 1
    sample_class[(meta_df['LibraryLayout_s'].values == layout) & 
                 (meta_df['infection_status_s'].values == 'Zika infected')] = 2
    platform = d_layout_platform[layout]
    sample_classes[platform] = sample_class

sample_classes['combined'] = sample_classes['MiSeq'] + sample_classes['NextSeq 500']
print sample_classes

for platform, sample_class in sample_classes.items():
    cd_res = geode.chdir(expr_df.values, sample_class, expr_df.index, 
                      gamma=.5, sort=False, calculate_sig=False)
    cd_coefs = np.array(map(lambda x: x[0], cd_res))
    cd_results[platform] = cd_coefs
    
    # sort CD in by absolute values in descending order
    srt_idx = np.abs(cd_coefs).argsort()[::-1]
    cd_coefs = cd_coefs[srt_idx][:600]
    sorted_DEGs = expr_df.index[srt_idx][:600]
    # split up and down
    up_genes = dict(zip(sorted_DEGs[cd_coefs > 0], cd_coefs[cd_coefs > 0]))
    dn_genes = dict(zip(sorted_DEGs[cd_coefs < 0], cd_coefs[cd_coefs < 0]))
    d_platform_cd[platform+'-up'] = up_genes
    d_platform_cd[platform+'-dn'] = dn_genes

print cd_results.head()

In [ ]:
## Check the cosine distance between the two signatures
from scipy.spatial.distance import cosine
from itertools import combinations
for col1, col2 in combinations(cd_results.columns, 2):
    print col1, col2, cosine(cd_results[col1], cd_results[col2])

In [ ]:
## Prepare count matrices
expect input data as obtained, e.g., from RNA-seq or another high-throughput sequencing experiment, in the form of a matrix of integer values. The value in the i-th row and the j-th column of the matrix tells how many reads (or fragments, for paired-end RNA-seq) have been assigned to gene i in sample j. Analogously, for other types of assays, the rows of the matrix might correspond e.g., to binding regions (with ChIP-Seq), species of bacteria (with metagenomic datasets), or peptide sequences (with quantitative mass spectrometry).

The values in the matrix should be counts of sequencing reads/fragments. This is important for DESeq2’s statistical model to hold, as only counts allow assessing the measurement precision correctly. It is important to never provide counts that were pre-normalized for sequencing depth/library size, as the statistical model is most powerful when applied to un-normalized counts, and is designed to account for library size differences internally.

## Align Reads to reference genome
The computational analysis of an RNA-seq experiment begins from the FASTQ files that contain the nucleotide sequence of each read and a quality score at each position. These reads must first be aligned to a reference genome or transcriptome, or the abundances and estimated counts per transcript can be estimated without alignment, as described above. In either case, it is important to know if the sequencing experiment was single-end or paired-end, as the alignment software will require the user to specify both FASTQ files for a paired-end experiment. The output of this alignment step is commonly stored in a file format called SAM/BAM.


## Define gene models

## Plot counts

## PCA Plot 

## Differential Expression Analysis
## Gene Clustering





In [ ]:
# Load expression matrix


In [ ]:
from diffexp.py_deseq import py_DESeq2

dds = py_DESeq2(count_matrix = df,
               design_matrix = sample_df,
               design_formula = '~ sample',
               gene_column = 'id') # <- telling DESeq2 this should be the gene ID column
    
dds.run_deseq() 
dds.get_deseq_result()
res = dds.deseq_result 
res.head()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(res.log2FoldChange, -np.log2(res.padj))

In [2]:
from dgeclust import CountData, SimulationManager

ModuleNotFoundError: No module named 'dgeclust'